# useMemo()

### IMPORTANT: React 19 introduced a new compiler which handles some performance problems by itself. It means that we don't have to use memo, useMemo, useCallback anymore!

Sometimes you have to calculate some value which is influencing optimization. After each re-render of the component React starts to calculate it again, even if it's not necessary.

For example:

In [ ]:
# SampleComponent.tsx

export default function SampleComponent() {
    
    const [counter, setCounter] = useState(1);
    
    # example function to simulate slow calculation
    function fib(x: number) {
        if (x <= 0) return 0;
        if (x == 1) return 1;
        return fib(x-1) + fib(x-2);
      }
    
    const exampleVariable = fib(40)
    
    return (
        <div>
            {counter}
            <button onClick={() => setCounter(prevState => prevState + 1)}>
                Add
            </button>
        </div>
    )
    
}

In above example we can increase the counter displayed on the screen. After each incrementation of counter exampleVariable will be re-calculated. To avoid this we can use useMemo() like in the example below:

In [ ]:
# SampleComponent.tsx

export default function SampleComponent() {
    
    const [counter, setCounter] = useState(1);
    
    # example function to simulate slow calculation
    function fib(x: number) {
        if (x <= 0) return 0;
        if (x == 1) return 1;
        return fib(x-1) + fib(x-2);
      }
    
    const exampleVariable = React.useMemo(() => {
        return fib(40);
    }, []) # we can add dependecy to trigger recalculation when the state is changed (for example array with fetched items), or
           # leave it blank to run it only at the first render.
    
    return (
        <div>
            {counter}
            <button onClick={() => setCounter(prevState => prevState + 1)}>
                Add
            </button>
        </div>
    )
    
}

# React.memo()

React.memo() allows to avoid re-render when the props are not changed. React renders all parent and child components even when the change is only in parent component.

In [ ]:
# ExampleParentComponent.tsx

export default function ExampleParentComponent() => {
    
    const [count, setCount] = useState(0);
    
    return (
        <div>
            This is count in parent {count}
            <button onClick={() => setCount(prevState => prevState + 1)}
            <ExampleChildComponent count={count} /> # we pass props
            <ExampleChildComponent /> # we don't pass any props
        </div>
    )
}

# ExampleChildComponent.tsx
export default function ExampleChildComponent({count}) {
    
    return (
        <div>
            Count in children component {count}
        </div>
    )
}

Everytime button is clicked, React re-renders parent and also a child component. To avoid this situation we have to use React.memo() hook.

In [ ]:
# ExampleChildComponent.tsx
function ExampleChildComponent({count}) { # remove export default 
    
    console.log("This component has been re-rendered") # you can use console.log to check if it's rendered
    
    return (
        <div>
            Count in children component {count}
        </div>
    )
}

# add here an export
export default React.memo(ExampleChildComponent)


React checks wether the props has changed or not. If yes, it re-renders the component. Otherwise it remains the same.

# React.useCallback()

What if prop is a function? Let's say we have a component that takes a function as a prop.

In [ ]:
# ExampleChildComponent.tsx
import { memo } from 'react'

function ExampleChildComponent({ todos, addTodo }) {
  console.log("child render");
  return (
    <>
      <h2>My Todos</h2>
      {todos.map((todo, index) => {
        return <p key={index}>{todo}</p>;
      })}
      <button onClick={addTodo}>Add Todo</button>
    </>
  );
};

export default memo(Todos) # we expect no re-render unless the props are changed.

In the parent component we pass the function.

In [ ]:
# ExampleParentComponent.tsx

import ExampleChildComponent from "./ExampleChildComponent";
import { useCallback, useState} from "react"
export default function Settings() {

  const [count, setCount] = useState(0);
  const [todos, setTodos] = useState([]);

  const increment = useCallback(() => {
    setCount((c) => c + 1);
  }, []);
    
  
  const addTodo = setTodos((t) => [...t, "New Todo"]); 
    # foo() is not equal to foo(), it means that it will always re-render child component. Even if only increment function
    # is called.

  return (
    <>
      <Todos todos={todos} addTodo={addTodo} />
      <hr />
      <div>
        Count: {count}
        <button onClick={increment}>+</button>
      </div>
    </>
  );
};


Despite no change of addTodo function the child component will re-render after clicking increment button. Why? Because props has changed? You can ask how? Function is not a primitive, so addTodo === addoTodo will return false. 

We can use useCallback hook to override it.

In [ ]:
# ExampleParentComponent.tsx

import ExampleChildComponent from "./ExampleChildComponent";
import { useCallback, useState} from "react"
export default function Settings() {

  const [count, setCount] = useState(0);
  const [todos, setTodos] = useState([]);

  const increment = useCallback(() => {
    setCount((c) => c + 1);
  }, []);
    
  
  const addTodo = useCallback(() => {  # use useCallback hook and set dependency, so it will be called only
      # when todos list is changed.
     setTodos((t) => [...t, "New Todo"]);
  }, [todos]);
    
  return (
    <>
      <Todos todos={todos} addTodo={addTodo} />
      <hr />
      <div>
        Count: {count}
        <button onClick={increment}>+</button>
      </div>
    </>
  );
};